In [4]:
import numpy as np
import re

In [1]:
test_input = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [3]:
import requests

URL = "https://adventofcode.com/{year}/day/{day}/input"
url = URL.format(year='2022', day=15)
cookies = {"session":"53616c7465645f5fd345d8c12076a357c8d31b2c0c4ad1c152eefbd5c1cf09e0c8d6d7f657fc558887d443d2d07fbc90728ada9b6a91afcecb9926ffb4981a93"}
req = requests.get(url, cookies=cookies)

if req.status_code != 200:
    print("Error. Got status:", req.status_code)
    print(req.text)
    exit(3)
else:
    my_input = req.text

In [220]:
class beacons :
    
    def __init__(self,this_input,look_row) :
        
        self.this_input = this_input
        self.look_row = look_row
        self.minX, self.minY, self.lenX, self.lenY = self.get_box() 
        self.c = self.get_empty_cave()
    
    def get_box(self) :
        
        allX = []
        allY = [self.look_row]
        for l in self.this_input.split("\n") :
            if l == "":
                continue
            m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
            for (x,y) in m :
                allX.append(int(x))
                allY.append(int(y))
        
        minx = min(allX)
        maxx = max(allX)
        miny = min(allY)
        maxy = max(allY)
        lenx = maxx-minx +1
        leny = maxy-miny +1
        return minx, miny, lenx, leny
        
    def x_to_i0(self,x) :
        if isinstance(x,str) :
            x=int(x)
        return x-self.minX
    
    def i0_to_x(self,i0) :
        if isinstance(i0,str) :
            i0=int(i0)
        return i0+self.minX
    
    def y_to_i1(self,y) :
        if isinstance(y,str) :
            y=int(y)
        return y-self.minY
    
    def i1_to_y(self,i1) :
        if isinstance(i1,str) :
            i1=int(i1)
        return i1+self.minY
    
    def get_empty_cave(self) :
        return np.zeros((self.lenX,self.lenY))
    
    def add_info(self) :
        
        for l in self.this_input.split("\n") :
            if l == "":
                continue
                
            m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
            Sx = int(m[0][0])
            Sy = int(m[0][1])
            Bx = int(m[1][0])
            By = int(m[1][1])
            
            I0 = self.x_to_i0(Sx)
            I1 = self.y_to_i1(Sy)
            self.c[(I0,I1)] = -10
            self.c[(self.x_to_i0(Bx),self.y_to_i1(By))] = -5
            
            d = abs(Sx-Bx) + abs(Sy-By)
            for i0 in range( 0,d+1) :
                dleft = d-abs(i0)

                for i1 in range(0,dleft+1) :
                        
                    b0m = I0 - i0
                    b1m = I1 - i1
                    if b0m == -1 :
                        self.c = np.r_[ np.zeros((1,self.lenY)) , self.c ]
                        self.minX -= 1
                        self.lenX += 1
                        I0 += 1
                        b0m += 1
                    if b1m == -1 :
                        self.c = np.c_[ np.zeros((self.lenX,1)) , self.c ]
                        self.minY -= 1
                        self.lenY += 1
                        I1 += 1
                        b1m += 1
                        
                    b0p = I0 + i0
                    b1p = I1 + i1
                    if b0p == self.lenX :
                        self.c = np.r_[ self.c , np.zeros((1,self.lenY)) ]
                        self.lenX += 1
                    if b1p == self.lenY :
                        self.c = np.c_[ self.c , np.zeros((self.lenX,1)) ]
                        self.lenY += 1
                        
                    if self.c[(b0p,b1p)] == 0 :
                        self.c[(b0p,b1p)] = 1
                    if self.c[(b0m,b1m)] == 0 :
                        self.c[(b0m,b1m)] = 1
                    if self.c[(b0m,b1p)] == 0 :
                        self.c[(b0m,b1p)] = 1
                    if self.c[(b0p,b1m)] == 0 :
                        self.c[(b0p,b1m)] = 1

    def res_part1(self):
        self.add_info()
        return sum(self.c[:,self.y_to_i1(self.look_row)]==1)

            
            
            
    def __str__(self):
        s = ""
        for l in self.c.transpose() :
            for el in l :
                if el == -10 :
                    s += "S"
                elif el == -5 :
                    s += "B"
                elif el == 1 :
                    s += "#"
                else :
                    s += "."
            s+="\n"
        return s

In [221]:
l = "Sensor at x=2, y=18: closest beacon is at x=-2, y=15"

In [222]:
c = beacons(test_input,10)

In [223]:
print(c)

............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................
............................



In [224]:
c.res_part1()

26

In [225]:
print(c)

..........#..........................
.........###.........................
........#####........................
.......#######.......................
......#########.............#........
.....###########...........###.......
....#############.........#####......
...###############.......#######.....
..#################.....#########....
.###################.#.###########...
##########S########################..
.###########################S#######.
..###################S#############..
...###################SB##########...
....#############################....
.....###########################.....
......#########################......
.......#########S#######S#####.......
........#######################......
.......#########################.....
......####B######################....
.....###S#############.###########...
......#############################..
.......#############################.
.......#############S#######S########
......B#############################.
.....#######

In [226]:
c = beacons(my_input,2000000)
c.res_part1()

MemoryError: Unable to allocate 166. TiB for an array with shape (4893543, 4665149) and data type float64

In [263]:
r = set()
yr = 18

l = "Sensor at x=2, y=18: closest beacon is at x=-2, y=15"
m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
Sx = int(m[0][0])
Sy = int(m[0][1])
Bx = int(m[1][0])
By = int(m[1][1])

d = abs(Sx-Bx) + abs(Sy-By)

In [264]:
if Sy-d <= yr and Sy+d >= yr :
    print("touch R")
    for i in range(d-abs(Sy-yr)+1) : 
        r.add(Sx+i)
        r.add(Sx-i)

touch R


In [265]:
r

{-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [266]:
known = set()
if Sy == yr :
    known.add(Sx)
if By == yr :
    known.add(Bx)

In [267]:
known

{2}

In [268]:
r-known

{-5, -4, -3, -2, -1, 0, 1, 3, 4, 5, 6, 7, 8, 9}

In [269]:
r = set()
known = set()
yr = 10
for l in test_input.split("\n") :
    if l == "":
        continue
    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)
    
    if Sy-d <= yr and Sy+d >= yr :
        for i in range(d-abs(Sy-yr)+1) : 
            r.add(Sx+i)
            r.add(Sx-i)
    if Sy == yr :
        known.add(Sx)
    if By == yr :
        known.add(Bx)
        
len(r-known)

26

In [272]:
r = set()
known = set()
yr = 2000000
for l in my_input.split("\n") :
    if l == "":
        continue
    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)
    
    if Sy-d <= yr and Sy+d >= yr :
        for i in range(d-abs(Sy-yr)+1) : 
            r.add(Sx+i)
            r.add(Sx-i)
    if Sy == yr :
        known.add(Sx)
    if By == yr :
        known.add(Bx)
        
len(r-known)
# 5942322 : too high

5240818

In [ ]:
for l in self.this_input.split("\n") :
    if l == "":
        continue

In [236]:
c = beacons(l,10)
c.add_info()
print(c)

...............
.......#.......
......###......
.....#####.....
....#######....
...B########...
..###########..
.#############.
#######S#######
.#############.
..###########..
...#########...
....#######....
.....#####.....
......###......
.......#.......



In [276]:
r = set(range(21))
r

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}

In [281]:
for yr in range(20+1):
    r = set(range(21))

    for l in test_input.split("\n") :
        if l == "":
            continue
        m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
        Sx = int(m[0][0])
        Sy = int(m[0][1])
        Bx = int(m[1][0])
        By = int(m[1][1])

        d = abs(Sx-Bx) + abs(Sy-By)
    
        if Sy-d <= yr and Sy+d >= yr :
            for i in range(d-abs(Sy-yr)+1) : 
                if Sx+i in r :
                    r.remove(Sx+i)
                if Sx-i in r :
                    r.remove(Sx-i)
        if len(r) == 0 :
#             print("not for ",yr)
            break
    if len(r) != 0 :
        print(list(r)[0],yr,list(r)[0]*4000000+yr)
        break

14 11 56000011


In [283]:
for yr in range(4000000+1):
    r = set(range(4000001))

    for l in my_input.split("\n") :
        if l == "":
            continue
        m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
        Sx = int(m[0][0])
        Sy = int(m[0][1])
        Bx = int(m[1][0])
        By = int(m[1][1])

        d = abs(Sx-Bx) + abs(Sy-By)
    
        if Sy-d <= yr and Sy+d >= yr :
            for i in range(d-abs(Sy-yr)+1) : 
                if Sx+i in r :
                    r.remove(Sx+i)
                if Sx-i in r :
                    r.remove(Sx-i)
        if len(r) == 0 :
#             print("not for ",yr)
            break
    if len(r) != 0 :
        print(list(r)[0],yr,list(r)[0]*4000000+yr)
        break

KeyboardInterrupt: 

In [285]:
LIM = 20

In [286]:
l = "Sensor at x=2, y=18: closest beacon is at x=-2, y=15"

In [292]:
m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
Sx = int(m[0][0])
Sy = int(m[0][1])
Bx = int(m[1][0])
By = int(m[1][1])

d = abs(Sx-Bx) + abs(Sy-By)

border = set()

for ix in range(0,d+2) :
    iy = d+1-ix
    print(ix,iy)
    bxp = Sx + ix
    bxm = Sx - ix
    byp = Sy + iy
    bym = Sy - iy

    if bxp <= LIM :
        if byp <= LIM :
#             print("\tpp : ",bxp,byp)
            border.add((bxp,byp))
        if bym >= 0 :
            border.add((bxp,bym))
#             print("\tpm : ",bxp,bym)
            
    if bxm >= 0 :
        if byp <= LIM :
            border.add((bxm,byp))
#             print("\t",bxm,byp)
        if bym >= 0 :
            border.add((bxm,bym))
#             print("\t",bxm,bym)
            
border

0 8
1 7
2 6
3 5
4 4
5 3
6 2
7 1
8 0


{(0, 12),
 (1, 11),
 (2, 10),
 (3, 11),
 (4, 12),
 (5, 13),
 (6, 14),
 (7, 15),
 (8, 16),
 (8, 20),
 (9, 17),
 (9, 19),
 (10, 18)}

In [311]:
all_borders = []

LIM = 20

for l in test_input.split("\n") :
    if l == "":
        continue

    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)

    border = set()

    for ix in range(0,d+2) :
        iy = d+1-ix
        bxp = Sx + ix
        bxm = Sx - ix
        byp = Sy + iy
        bym = Sy - iy

        if bxp <= LIM :
            if byp <= LIM :
                border.add((bxp,byp))
            if bym >= 0 :
                border.add((bxp,bym))

        if bxm >= 0 :
            if byp <= LIM :
                border.add((bxm,byp))
            if bym >= 0 :
                border.add((bxm,bym))
                
                
    all_borders.append(border)



In [320]:
set_test_Points = set()
for i in range(len(all_borders)-1):
    for ii in range(1,len(all_borders)) :
        set_test_Points.update(set.intersection(all_borders[i],all_borders[ii]))

In [321]:
len(set_test_Points)

124

In [305]:
def test_Point(x,y,this_input) :
    for l in this_input.split("\n") :
        
        if l == "":
            continue
            
        m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
        Sx = int(m[0][0])
        Sy = int(m[0][1])
        Bx = int(m[1][0])
        By = int(m[1][1])

        dist_B = abs(Sx-Bx) + abs(Sy-By)
        dist_P = abs(Sx-x) + abs(Sy-y)
        if dist_P <= dist_B :
            return False
    return True

In [322]:
for x,y in set_test_Points :
    if test_Point(x,y,test_input):
        print(x,y,x*4000000+y)
        break
        

14 11 56000011


In [307]:
all_borders = set()

LIM = 4000000

for l in my_input.split("\n") :
    if l == "":
        continue

    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)

    border = set()

    for ix in range(0,d+2) :
        iy = d+1-ix
        bxp = Sx + ix
        bxm = Sx - ix
        byp = Sy + iy
        bym = Sy - iy

        if bxp <= LIM :
            if byp <= LIM :
                all_borders.add((bxp,byp))
            if bym >= 0 :
                all_borders.add((bxp,bym))

        if bxm >= 0 :
            if byp <= LIM :
                all_borders.add((bxm,byp))
            if bym >= 0 :
                all_borders.add((bxm,bym))




In [308]:
len(all_borders)

59206204

In [309]:
for x,y in all_borders :
    if test_Point(x,y,my_input):
        print(x,y,x*4000000+y)
        break
        

KeyboardInterrupt: 

In [323]:
all_borders = []

LIM = 4000000

for l in my_input.split("\n") :
    if l == "":
        continue

    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)

    border = set()

    for ix in range(0,d+2) :
        iy = d+1-ix
        bxp = Sx + ix
        bxm = Sx - ix
        byp = Sy + iy
        bym = Sy - iy

        if bxp <= LIM :
            if byp <= LIM :
                border.add((bxp,byp))
            if bym >= 0 :
                border.add((bxp,bym))

        if bxm >= 0 :
            if byp <= LIM :
                border.add((bxm,byp))
            if bym >= 0 :
                border.add((bxm,bym))
                
                
    all_borders.append(border)



In [325]:
len(all_borders)

33

In [327]:
set_test_Points = set()
for i in range(len(all_borders)-2):
    for ii in range(i,len(all_borders)-1) :
        for iii in range(ii,len(all_borders)):
            set_test_Points.update(set.intersection(all_borders[i],all_borders[ii],all_borders[iii]))

In [328]:
len(set_test_Points)

57901989

In [305]:
def test_Point(x,y,this_input) :
    for l in this_input.split("\n") :
        
        if l == "":
            continue
            
        m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
        Sx = int(m[0][0])
        Sy = int(m[0][1])
        Bx = int(m[1][0])
        By = int(m[1][1])

        dist_B = abs(Sx-Bx) + abs(Sy-By)
        dist_P = abs(Sx-x) + abs(Sy-y)
        if dist_P <= dist_B :
            return False
    return True

In [329]:
for x,y in set_test_Points :
    if test_Point(x,y,my_input):
        print(x,y,x*4000000+y)
        break
        

3303271 2906101 13213086906101


In [330]:
capteurs = []

for l in my_input.split("\n") :
    if l == "":
        continue

    m = re.findall("x=(?P<Xs>-?\d*), y=(?P<Ys>-?\d*)",l)
    Sx = int(m[0][0])
    Sy = int(m[0][1])
    Bx = int(m[1][0])
    By = int(m[1][1])

    d = abs(Sx-Bx) + abs(Sy-By)
    
    capteurs.append((Sx,Sy,d))


In [331]:
capteurs

[(2899860, 3122031, 619340),
 (1836719, 1116779, 1083557),
 (3995802, 2706630, 397919),
 (2591204, 2008272, 507531),
 (2546593, 1538222, 971316),
 (252214, 61954, 1587428),
 (950, 1106672, 1064820),
 (1349445, 1752783, 934827),
 (3195828, 3483667, 186685),
 (2057761, 2154359, 175065),
 (2315350, 3364640, 564059),
 (327139, 2426600, 1635906),
 (3943522, 2854345, 199956),
 (3358620, 516881, 1827302),
 (1788376, 8679, 1399613),
 (3344883, 3537985, 16688),
 (2961064, 2697125, 551182),
 (3780090, 2093546, 672107),
 (3291917, 3398703, 175560),
 (3999864, 2998005, 110606),
 (2919272, 3732950, 408173),
 (2057404, 2947435, 967784),
 (1072126, 645784, 1351346),
 (3549465, 2554712, 597582),
 (3550313, 3121694, 462634),
 (3405149, 3483630, 118385),
 (2600212, 3961193, 519470),
 (1102632, 3932527, 1476632),
 (67001, 3506079, 296289),
 (3994250, 3975025, 971452),
 (3019750, 2125144, 361701),
 (3282319, 3656404, 177219),
 (2797371, 3645126, 198448)]

In [332]:
def result() :
    LIM = 4000000

    for (Sx,Sy,d) in capteurs:

        for ix in range(0,d+2) :
            iy = d+1-ix
            bxp = Sx + ix
            bxm = Sx - ix
            byp = Sy + iy
            bym = Sy - iy

            if bxp <= LIM :
                if byp <= LIM :
                    if test_Point(bxp,byp,my_input):
                        print(bxp,byp,bxp*4000000+byp)
                        return True
                if bym >= 0 :
                    if test_Point(bxp,bym,my_input):
                        print(bxp,bym,bxp*4000000+bym)
                        return True

            if bxm >= 0 :
                if byp <= LIM :
                    if test_Point(bxm,byp,my_input):
                        print(bxm,byp,bxm*4000000+byp)
                        return True
                if bym >= 0 :
                    if test_Point(bxm,bym,my_input):
                        print(bxm,bym,bxm*4000000+bym)
                        return True

In [333]:
result()

3303271 2906101 13213086906101


True

In [ ]:
13213086906101